构建一个 RAG 问答链

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

# 向量检索器
db = FAISS.load_local("your_index", OpenAIEmbeddings())

# 构建 LLM 问答链（RAG）
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    retriever=db.as_retriever(),
    chain_type="stuff"
)

response = qa_chain.run("What is LangChain?")
print(response)


LangChain Agent

In [ ]:
from langchain.agents import initialize_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

search = DuckDuckGoSearchRun()

agent = initialize_agent(
    tools=[search],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("Who is the CEO of OpenAI?")
print(response)


使用 LangChain 构建带记忆的 Agent

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.memory import CombinedMemory

# 初始化 LLM
llm = ChatOpenAI(temperature=0)

# 构建对话记忆模块
buffer_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
summary_memory = ConversationSummaryMemory(llm=llm, memory_key="summary")

# 组合两种记忆
combined_memory = CombinedMemory(memories=[summary_memory, buffer_memory])

# 构建对话链
conversation = ConversationChain(
    llm=llm,
    memory=combined_memory,
    verbose=True
)


ReAct（推理和行动） Agent 

In [ ]:
#定义搜索函数并封装成 Tool
from langchain.tools import Tool
from duckduckgo_search import DDGS  # 使用 DuckDuckGo 搜索（免费）

# 自定义搜索函数
def web_search(query: str) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join([r["body"] for r in results]) if results else "No results found."

# 封装为 Tool
search_tool = Tool.from_function(
    func=web_search,
    name="Search",
    description="Useful for answering factual or current event questions from the web."
)

#构建 ReAct Agent
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4")  # 可换成 gpt-3.5-turbo

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",  # 使用 ReAct agent 模式
    verbose=True
)

#运行一个问题测试
response = agent.run("What is the ReAct framework in LLMs?")
print(response)
